# Federated Statistics

Before training the model, data scientists need to understand the data distribution and data quality of the data at different sites. As we can't access the raw data, we need to perform federated Analytics to get the statistics of the data at global level. That's where Federated Statistics comes in.


## Objective
NVIDIA FLARE will provide built-in federated statistics operators (controllers and executors) that 
can generate global statistics based on local client side statistics.

At each client site, we could have one or more datasets (such as "train" and "test" datasets); each dataset may have many 
features. For each feature in the dataset, we will calculate the statistics and then combine them to produce 
global statistics for all the numeric features. The output would be complete statistics for all datasets in clients and global.    

The statistics here are commonly used statistics: count, sum, mean, std_dev and histogram for the numerical features.
The max, min are not included as it might violate the client's data privacy. The mean will be calculated with count and sum. 

The quantile can also be included, but requires additional dependency to be installed. We are not going to include it in the default installation.


A client will only need to implement the selected methods of "Statistics" class from statistics_spec.

The result will be statistics for all features of all datasets at all sites as well as global aggregates. 
The result could be visualized via the visualization utility in the notebook. 


## Assumptions
 
Assume that clients will provide the following: 
   * users need to provide target statistics such as count, histogram etc. of targeted statistics features
   * users need to provide the local statistics for the **target statistics** (by implementing the statistic_spec), not all statistics. For example, if the target statistics does not include histogram, then users will not need to provide the histogram calculation function.
   * users need to provide the datasets and dataset features (feature name, data type)
   * Note: count is always required as we use count to enforce data privacy policy
   
We only support **numerical features**, not categorical features. But users can return all types of features;
the non-numerical features will be removed.



## Statistics

  Federated statistics includes numerics statistics measures for 
  * count
  * mean 
  * sum
  * std_dev
  * histogram 
  * quantile
    
  We did not include min, max value to avoid data privacy concern. 


#### Quantile

Quantile statistics refers to statistical measures that divide a probability distribution or dataset into intervals with equal probabilities or proportions. Quantiles help summarize the distribution of data by providing key points that indicate how values are spread.

##### Key Quantiles:
1. Median (50th percentile): The middle value of a dataset, dividing it into two equal halves.
2. Quartiles (25th, 50th, 75th percentiles): Divide the data into four equal parts:
* Q1 (25th percentile): Lower quartile, below which 25% of the data falls.
* Q2 (50th percentile): Median.
* Q3 (75th percentile): Upper quartile, below which 75% of the data falls.
3. Deciles (10th, 20th, ..., 90th percentiles): Divide the data into ten equal parts.
4. Percentiles (1st, 2nd, ..., 99th): Divide the data into 100 equal parts.

##### Usage of Quantiles:
* Descriptive Statistics: Summarizes the spread of data.
* Outlier Detection: Helps identify extreme values.
* Machine Learning: Used in feature engineering, normalization, and decision tree algorithms.
* Risk Analysis: Used in finance (e.g., Value at Risk, VaR).

 
## Privacy Policy and Privacy Filters

NVFLARE provide data privacy protection through privacy filters [privacy-management](https://nvflare.readthedocs.io/en/main/user_guide/security/site_policy_management.html#privacy-management)
Each site can have its own privacy policy. 

### Local privacy policy

privacy.json provides local site specific privacy policy.
The policy is likely setup by the company and implemented by organization admin
for the project. For different type of scope or categories, there are might be different types of policies. 

### Privacy configuration

The NVFLARE privacy configuration consists of set of task data filters and task result filters
* The task data filter applies before client executor executes;
* The task result filter is applied after the client executor executes and before the data is sent to the server;
* Both the data filter and result filter are grouped by scope.

Each job will need to have a privacy scope. If not specified, the default scope will be used. If default scope is not
defined and job doesn't specify the privacy scope, the job deployment will fail, and job will not executed

### Privacy Policy Instrumentation 

There are different ways to set privacy filters depending on the use cases


####  Set Privacy Policy as researcher

You can specify the "task_result_filters" in config_fed_client.json to specify
the privacy control.  This is useful when you develop these filters

#### Setup site privacy policy as org admin

Once the company decides to instrument certain privacy policy independent of individual
job, one can copy the local directory privacy.json content to clients' local privacy.json ( merge not overwrite).
in this example, since we only has one app, we can simply copy the private.json from local directory to

* site-1/local/privacy.json
* site-2/local/privacy.json

We need to remove the same filters from the job definition in config_fed_client.json
by simply set the "task_result_filters" to empty list to avoid **double filtering**
```
"task_result_filters": []
```
#### Job filter vs filters in private.json

Privacy filters are defined within a privacy scope.
If a job's privacy scope is defined or has default scope, then the scope's filters (if any) are applied
before the job-specified filters (if any). This rule is enforced during task execution time.

With such rules, if we have both task result filters and privacy scoped filters, we need to understand
that the privacy filters will be applied first, then the job filters. The filters will be applied by their specified orders

### Statistics Privacy Filters
Statistics privacy filters are task result filters. We already built one for Statistics. 

```
StatisticsPrivacyFilter
```
The StatisticsPrivacyFilter consists of several `StatisticsPrivacyCleanser`s focused on the statistics sent
from client to server.

`StatisticsPrivacyCleanser` can be considered as an interceptor before the results delivered to server.
Currently, we use three `StatisticsPrivacyCleanser`s to guard the data privacy. The reason we built 
`StatisticsPrivacyCleanser` instead of separate filters is to avoid repeated data de-serialization.

#### MinCountCleanser:
Check against the number of count returned from client for each dataset and each feature.

If the min_count is not satisfied, there is potential risk of reveal client's real data. Then remove that feature's statistics
from the result for this client.

#### HistogramBinsCleanser:
For histogram calculations, number of bins can't be too large compare to count. if the bins = count, then
we also reveal the real data. This check to make sure that the number of bins be less than X percent of the count.
X = max_bins_percent in percentage, for 10 is for 10%
if the number of bins for the histogram is not satisfy this specified condition, the resulting histogram will be removed
from statistics before sending to server.

#### AddNoiseToMinMax
For histogram calculations, if the feature's histogram bin's range is not specified, we will need to use local data's min
and max values to calculate the global min/max values, then use the global min, max values as the bin range for histogram
calculation. But send the server the local min, max values will reveal client's real data.
To protect data privacy, we add noise to the local min/max values.

Min/max random is used to generate random noise between (min_noise_level and max_noise_level).
for example, the random noise is to be within (0.1 and 0.3),i.e. 10% to 30% level. These noise
will make local min values smaller than the true local min values, and max values larger than
the true local max values. As result, the estimate global max and min values (i.e. with noise)
are still bounded the true global min/max values, in such that
```
est. global min value <
    true global min value <
        client's min value <
            client's max value <
                true global max <
                        est. global max value
```


## Summary

We provided federated statistics operators that can easily aggregate and visualize the local statistics for
different data site and features. We hope this feature will make it easier to perform federated data analysis.  

We provide several examples to demonstrate how should the operators be used. 

The main steps are 
* provide server side configuration to specify target statistics and their configurations and output location
* implement the local statistics generator (statistics_spec)
* provide client side configuration to specify data input location
 

Now lets dive into the examples


* [Federated Statistics with tabular data](./federated_statistics_with_tabular_data/federated_statistics_with_tabular_data.ipynb) demonstrates how to create federated statistics for data that can be represented as Pandas DataFrames.
* [Federated Statistics with image data](./federated_statistics_with_image_data/federated_statistics_with_image_data.ipynb) shows how to compute local and global image statistics with the consideration that data is private at each of the client sites.
 